In [ ]:
#| default_exp core

# core

> Fill in a module description here

In [ ]:
#| export
from fastcore.utils import *
from fastcore.xml import *
from fasthtml.common import *
from fasthtml.svg import *
from json import loads
import pathlib

In [ ]:
__file__='../fastlucide/_icons.py'

In [ ]:
#| export
def read_icons():
    path = pathlib.Path(__file__).parent
    fpath = path/'_icons.py'
    return loads(fpath.read_text())

In [ ]:
icons = read_icons()

In [ ]:
nm = 'a-arrow-down'
ico = icons[nm]
ico

[['path', {'d': 'M3.5 13h6'}],
 ['path', {'d': 'm2 16 4.5-9 4.5 9'}],
 ['path', {'d': 'M18 7v9'}],
 ['path', {'d': 'm14 12 4 4 4-4'}]]

In [ ]:
svg_attrs = dict(fill='none', stroke='currentColor', stroke_width='2', stroke_linecap='round', stroke_linejoin='round')

In [ ]:
#| export
def sz_attrs(sz, vbox=24):
    if isinstance(vbox, int): vbox = (vbox, vbox)
    if isinstance(sz, int): sz = (sz, sz)
    return dict(viewBox=f'0 0 {vbox[0]} {vbox[1]}', width=f'{sz[0]}px', height=f'{sz[1]}px')

In [ ]:
sym = [ft(t, **attrs) for t,attrs in ico]
show(Svg(*sym, **sz_attrs(24), **svg_attrs))
show(Svg(*sym, **sz_attrs(16), **svg_attrs))

In [ ]:
#| export
def symbol(
    icons, # icon dict
    nm, # Name of icon in lucide
    pre='' # Prefix to add to element id
):
    "Create a `symbol` element for an icon"
    ico = icons[nm]
    sym = [ft(t, **attrs) for t,attrs in ico]
    return Symbol(*sym, id=pre+nm)

In [ ]:
symbol(icons, nm, 'l-')

```html
<symbol id="l-a-arrow-down"><path d="M3.5 13h6"></path><path d="m2 16 4.5-9 4.5 9"></path><path d="M18 7v9"></path><path d="m14 12 4 4 4-4"></path></symbol>
```

In [ ]:
#| export
def sprites(
    icons, # icon dict
    nms, # List of lucide icon names
    pre='' # Prefix to add to all element ids
):
    "SVG element containing all symbols in `nms`"
    syms = [symbol(icons, nm, pre) for nm in nms]
    return Svg(Defs(*syms), style="display: none")

In [ ]:
sps = sprites(icons, [nm, 'accessibility'], 'l-')
show(sps)

In [ ]:
#| export
def _style_str(stroke=None, fill=None):
    "Build CSS style string from stroke/fill"
    styles = []
    if stroke: styles.append(f'stroke: {stroke}')
    if fill: styles.append(f'fill: {fill}')
    return '; '.join(styles) if styles else None

In [ ]:
#| export
def icon(
    nm, # Name of icon in lucide
    pre='', # Prefix to add to element id
    cls="lucide-icon", # class to use for svg
    vbox=24, # viewBox size (int for square, or (w,h) tuple)
    sz=24, # size of svg
    stroke=None, # Stroke CSS
    fill=None, # Fill CSS
    **attrs # additional attrs for svg
):
    "A `use` element in an `svg` element refering to `nm`"
    style = _style_str(stroke, fill)
    return Svg(Use(href=f"#{pre}{nm}"), cls=cls, icon=nm, style=style, **sz_attrs(sz, vbox), **attrs)

In [ ]:
used = icon('accessibility', 'l-', sz=16, **svg_attrs)
used

```html
<svg xmlns="http://www.w3.org/2000/svg" viewbox="0 0 24 24" height="16px" width="16px" icon="accessibility" stroke-width="2" stroke-linecap="round" stroke-linejoin="round" class="lucide-icon" style="stroke: currentColor; fill: none"><use href="#l-accessibility"></use></svg>
```

In [ ]:
show(used)

In [ ]:
#| export
def SvgStyle(cls="lucide-icon"):
    "Styles required for lucide icons to display correctly"
    return Style(f'.{cls} {{ stroke: currentColor; fill: none; stroke-width: 2; stroke-linecap: round; stroke-linejoin: round; }}')

In [ ]:
#| export
class SvgSprites:
    "Create an track used icons"
    def __init__(self, pre='', vbox=24, sz=24, cls="lucide-icon", nms=(), **attrs):
        nms = set(nms)
        self.attrs = attrs
        self.icons = read_icons()
        store_attr()

    def __call__(self, nm, sz=None, cls="", **kw):
        self.nms.add(nm)
        if not sz: sz=self.sz
        attrs = self.attrs | kw
        return icon(nm, self.pre, vbox=self.vbox, sz=sz, cls = f"{self.cls} {cls}", **attrs)

    def __ft__(self):
        return SvgStyle(cls=self.cls),sprites(self.icons, self.nms, self.pre)

In [ ]:
ss = SvgSprites('lc-', sz=18)
ss(nm)

```html
<svg xmlns="http://www.w3.org/2000/svg" viewbox="0 0 24 24" height="18px" width="18px" icon="a-arrow-down" class="lucide-icon "><use href="#lc-a-arrow-down"></use></svg>
```

In [ ]:
show(ss('a-arrow-down'))
show(ss('accessibility'))
show(ss)

In [ ]:
show(Style('.hover-bg:hover { background-color: aliceblue; }'))
show(ss('accessibility', sz=30, cls='hover-bg'))

In [ ]:
show(ss('accessibility', stroke='blue'))

In [ ]:
show(Div(ss('accessibility', stroke='blue'), style='background-color: aliceblue'))

In [ ]:
#| export
@patch
def combined(self:SvgSprites, nm1, nm2, position='overlap', sz=None, 
             stroke1=None, fill1=None, stroke2=None, fill2=None, **kw):
    "Combine two icons with positioning"
    self.nms.add(nm1)
    self.nms.add(nm2)
    if sz is None: sz = self.sz
    vbox = self.vbox
    
    href1, href2 = f"#{self.pre}{nm1}", f"#{self.pre}{nm2}"
    style1, style2 = _style_str(stroke1, fill1), _style_str(stroke2, fill2)
    
    trans = ''
    if position == 'overlap':
        szx = szy = sz
        vx = vy = vbox
    elif position == 'beside':
        szx = 2 * (szy := sz)
        vx = 2 * (vy := vbox)
        trans = f'translate({vbox}, 0)'
    else:
        szy = 2 * (szx := sz)
        vy = 2 * (vx := vbox)
        trans = f'translate(0, {vbox})'

    use2_kw = {'transform': trans} if trans else {}
    return Svg(Use(href=href1, style=style1),
               Use(href=href2, style=style2, **use2_kw),
               cls=self.cls, icon=f'{nm1}-{nm2}', **sz_attrs((szx, szy), (vx, vy)), **kw)

In [ ]:
ss = SvgSprites('lc-', sz=32)
show(ss.combined('circle', 'x', stroke1='steelblue', stroke2='crimson'))
show(ss.combined('circle', 'x', position='beside', stroke1='steelblue', stroke2='crimson'))
show(ss)

## export -

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()